# PySparkに慣れる

## インストール
基本は[Quickstart](https://spark.apache.org/docs/latest/api/python/getting_started/testing_pyspark.html)に従って、  
pip install pysparkすればインストールできる。  
ただし、jdkは入っていない場合はjdkもインストールしておかないとJAVA_HOMEが設定されていないというエラーが起きる。  

## 動作テスト
文法はPolarsに似ている。（というかPolarsがPySparkに似ている？）  
ノートPCでの動作はPolarsよりも遅く感じる。

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a SparkSession。pythonからsparkを使う場合、セッションの作成が必要。
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/03 15:10:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sample_data = [{"name": "John    D.", "age": 30},
  {"name": "Alice   G.", "age": 25},
  {"name": "Bob  T.", "age": 35},
  {"name": "Eve   A.", "age": 28}]

df = spark.createDataFrame(sample_data)

In [3]:
# from pyspark.sql import functions as Fで読み込んだりもする
from pyspark.sql.functions import col, regexp_replace

# Remove additional spaces in name
def remove_extra_spaces(df, column_name):
    # Remove extra spaces from the specified column
    df_transformed = df.withColumn(column_name, regexp_replace(col(column_name), "\\s+", " "))

    return df_transformed

transformed_df = remove_extra_spaces(df, "name")

transformed_df.show()

+---+--------+
|age|    name|
+---+--------+
| 30| John D.|
| 25|Alice G.|
| 35|  Bob T.|
| 28|  Eve A.|
+---+--------+



## 色々お試し

### 下記のような読み込み方はダメ

In [ ]:
sample_data_2 = {
    "name" : ["John    D.", "Alice   G.", "Bob  T.", "Eve   A."],
    "age" : [30, 25, 35, 28]
}
# [{"name": "John    D.", "age": 30},
#   {"name": "Alice   G.", "age": 25},
#   {"name": "Bob  T.", "age": 35},
#   {"name": "Eve   A.", "age": 28}]

aaa = spark.createDataFrame(sample_data_2)

PySparkTypeError: [CANNOT_INFER_SCHEMA_FOR_TYPE] Can not infer schema for type: `str`.

### 列を足す

In [5]:
# 列を足す。
transformed_df.withColumn(
    "new_column_1", col("age") + 3
).show()

+---+--------+------------+
|age|    name|new_column_1|
+---+--------+------------+
| 30| John D.|          33|
| 25|Alice G.|          28|
| 35|  Bob T.|          38|
| 28|  Eve A.|          31|
+---+--------+------------+



### フィルタリング

In [8]:
transformed_df.filter(
    col("age") >= 30
).show()

+---+-------+
|age|   name|
+---+-------+
| 30|John D.|
| 35| Bob T.|
+---+-------+

